In [3]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time
import ast

In [4]:
testes = pd.read_excel('../docs/testes.xlsx')

acesso negado

In [4]:
def captura_num_processos(filtrados):
    copia_filtrados = filtrados.copy()
    sucessos = 0
    processos = []

    for i in range(copia_filtrados.shape[0]):
        link = copia_filtrados.iloc[i, 3]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        num_processo = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            num_processo = match.group(0)
            sucessos += 1

        processos.append(num_processo)
    
    print(f"Números de processos capturados: {sucessos}")

    copia_filtrados["num_processo"] = processos
    return copia_filtrados
    
filtrados_com_num = captura_num_processos(testes)
# filtrados_com_num.to_excel("processos_fitrados.xlsx")
filtrados_com_num

KeyboardInterrupt: 

In [ ]:
processos_nao_encontrados = filtrados_com_num.loc[filtrados_com_num['num_processo'] == "Não encontrado", :]

In [16]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 3]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto)
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = processos_nao_encontrados[processos_nao_encontrados['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)

In [17]:
acessos_negados

,CNJ,processoID,processoAnexoID,Download copia,Download original,Tipo de anexo,Publicado em,Obtido em,Tribunal,Titulo,num_processo


Dentre os testes, nenhum acesso negado

# Utilização de Gemini 

Testar a validade de uso do Gemini a partir de uma amostra de testes da base de dados do jusbrasil.

In [ ]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

filtrados = testes
response = requests.get(filtrados.iloc[1,3])
response.encoding = 'utf-8'
content_type = response.headers.get('Content-Type', '')
texto_extraido = ""

if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(response.content)
        tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
    with pymupdf.open(tmp_path) as doc:
        for page in doc:
            texto_extraido += page.get_text()
    
elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
    soup = BeautifulSoup(response.content, 'html.parser')
    texto_extraido = soup.get_text()

# Prompt
prompt = f"""Você está recebendo um texto bruto de um documento sobre
        algum processo judiciário. O seu interesse principal é descobrir se este texto trata de um caso relacionado à 
        valoração de danos ambientais.
        Assim, a partir das informações nele contidas, devolva APENAS UMA LISTA (NÃO DEVOLVA O TEXTO) 
        com as seguintes informações em ordem:
        Elemento 0 da lista: Texto fala de um processo em que houve algum tipo de dano ambiental (True ou False)
        Elemento 1 da lista: Texto menciona uma georreferência do local afetado pelo dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 2 da lista: A UF do local ou seja "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará",
    "Distrito Federal", "Espírito Santo", "Goiás", "Maranhão",
    "Mato Grosso", "Mato Grosso do Sul", "Minas Gerais", "Pará",
    "Paraíba", "Paraná", "Pernambuco", "Piauí", "Rio de Janeiro",
    "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", "Roraima",
    "Santa Catarina", "São Paulo", "Sergipe", "Tocantins" ou variações (devolva a string
        "Não encontrado" caso não exista)
        Elemento 3 da lista: O município ou cidade do local do processo (devolva a string
        "Não encontrado" caso não exista). 
        Elemento 4 da lista: O responsável pelo dano ambiental - aqui pode ser uma empresa ou pessoa física.
        (devolva a string "Não encontrado" caso não exista)
        Elemento 5 da lista: Categoria do elemento 4 - "Pessoa Física" ou "Pessoa Jurídica"
        (devolva a string "Não encontrado" caso não exista)
        Elemento 6 da lista: A data de ocorrência do impacto ambiental no formato DD/MM/AA
        (devolva a string "Não encontrado" caso não exista)
        Elemento 7 da lista: A extensão da área afetada pelo impacto ambiental
        (devolva a string "Não encontrado" caso não exista)
        Elemento 8 da lista: A unidade de medida do elemento 7
        (devolva a string "Não encontrado" caso não exista)
        Elemento 9 da lista: A metodologia de punição do responsável pelo impacto
        (devolva a string "Não encontrado" caso não exista)
        Elemento 10 da lista: O valor da multa que o responsável deve pagar pelas suas ações (em R$)
        (devolva a string "Não encontrado" caso não exista)
        Texto bruto: {texto_extraido}"""
generation_config_low_temp = genai.GenerationConfig(
    max_output_tokens=500,
    temperature=0.0,
)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
time.sleep(3)
resposta.text

"```\n[False, 'Não encontrado', 'Maranhão', 'Paço do Lumiar', 'BRK AMBIENTAL - MARANHÃO S.A', 'Pessoa Jurídica', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado', 'Não encontrado']\n```\n"

## Teste do Gemini para algumas amostras

In [6]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)


In [ ]:
respostas = {}
filtrados = testes


for i in range(0,10):
    response = requests.get(filtrados.iloc[i,3])

    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    texto_extraido = ""

    if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto_extraido += page.get_text()
    
    elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
        soup = BeautifulSoup(response.content, 'html.parser')
        texto_extraido = soup.get_text()

# Prompt
    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] O texto trata de dano ambiental?
     Valores possíveis: True ou False

    [1] Trecho do texto que justifique a resposta acima.

    [2] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [3] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [4] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [5] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [6] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [7] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [8] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [9] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [10] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [11] Valor da multa, compensação monetária ou indenização atribuída (em reais, formato: R$ 000.000,00)
     Se não houver, retorne: "Não encontrado valor"

    [12] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,filtrados.iloc[i,1])
    lista = [x for x in lista if x != '']

    respostas[filtrados.iloc[i,2]] = lista
print(respostas)


{2643544408: [652049379, 'False', 'O texto trata de um processo judicial sobre cobranças de água consideradas abusivas pela consumidora, devido a divergências entre o consumo registrado pelo hidrômetro e o cobrado em faturas. Não há menção a danos ambientais.', 'Não encontrado georreferencia', 'Maranhão', 'Paço do Lumiar', 'BRK Ambiental Maranhão S.A', 'Pessoa Jurídica', 'Não encontrado data', '150', 'm²', 'Laudo pericial', 'Não encontrado valor', 'Não encontrado compensacao'], 2643544419: [652049379, 'False', 'Não encontrado trecho', 'Não encontrado georreferencia', 'Maranhão', 'Não encontrado cidade', 'BRK AMBIENTAL - MARANHAO S.A', 'Pessoa Jurídica', 'Não encontrado data', 'Não encontrado area', 'Não encontrado medida', 'Não encontrado metodo', 'R$ 15.000,00', 'Não encontrado compensacao'], 2643544426: [652049379, 'False', 'O texto trata de um processo judicial sobre cobranças de água consideradas abusivas pela consumidora, devido a valores altos em suas faturas. Não há menção a dan

In [80]:
#print((list(respostas.keys())))
respostas_teste = pd.DataFrame.from_dict(respostas, orient='index')

respostas_teste.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria']

In [81]:
display(respostas_teste)

,processoAnexoID,processo_ambiental,justificativa,georreferência,UF,municipio,responsavel,tipo_responsavel,data,área,unidade,metodologia_punição,valor_multa,compensacao_nao_monetaria
2643544408,652049379,False,O texto trata de um processo judicial sobre co...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Laudo pericial,Não encontrado valor,Não encontrado compensacao
2643544419,652049379,False,Não encontrado trecho,Não encontrado georreferencia,Maranhão,Não encontrado cidade,BRK AMBIENTAL - MARANHAO S.A,Pessoa Jurídica,Não encontrado data,Não encontrado area,Não encontrado medida,Não encontrado metodo,"R$ 15.000,00",Não encontrado compensacao
2643544426,652049379,False,O texto trata de um processo judicial sobre co...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Processo judicial,Não encontrado valor,Não encontrado compensacao
590939495,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado area,Não encontrado medida,Não encontrado metodo,Não encontrado valor,"Plano de recuperação para conter a erosão, cer..."
590939496,612465137,True,AÇÃO CIVIL PÚBLICA CÍVEL (65),Não encontrado georreferencia,Não encontrado UF,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado data,Não encontrado area,Não encontrado medida,Não encontrado metodo,Não encontrado valor,Não encontrado compensacao
590939497,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado area,Não encontrado medida,Não encontrado metodo,Não encontrado valor,"Plano de recuperação para conter a erosão, cer..."
1186042345,582564055,False,Não encontrado,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Não encontrado responsavel,Não encontrado pessoa,Não encontrado data,Não encontrado area,Não encontrado medida,Não encontrado metodo,Não encontrado valor,Não encontrado compensacao
2589045924,690025576,True,O presente Laudo Técnico Pericial tem como obj...,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,CARMENSE COMERCIAL LTDA,Pessoa Jurídica,04/05/2023,Não encontrado area,Não encontrado medida,Processo Judicial Eletrônico,"R$ 3.807.896,40",desativação e a desmobilização completa do par...
2589045929,690025576,True,Mancha de óleo sobre o solo – Risco de contami...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Carmense Comercial Ltda,Pessoa Jurídica,Não encontrado data,Não encontrado area,Não encontrado medida,Não encontrado metodo,Não encontrado valor,Não encontrado compensacao
2589045772,690025576,True,ÁREA DE ATUAÇÃO: Meio Ambiente,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,Carmense Comercial Ltda,Pessoa Jurídica,04/05/2023,Não encontrado area,Não encontrado medida,Processo Judicial Eletrônico,Não encontrado valor,Investigação Confirmatória em todo o parque in...


## Para 1-50 processos

In [21]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

filtrados = testes
respostas_camila = {}


In [22]:
def analise(i,respostas):
    response = requests.get(filtrados.iloc[i,3])

    response.encoding = 'utf-8'
    content_type = response.headers.get('Content-Type', '')
    texto_extraido = ""

    if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
        with pymupdf.open(tmp_path) as doc:
            for page in doc:
                texto_extraido += page.get_text()
    
    elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
        soup = BeautifulSoup(response.content, 'html.parser')
        texto_extraido = soup.get_text()

# Prompt
    prompt = f"""
    Tarefa: Você receberá um texto bruto que descreve um processo judicial. Seu objetivo é analisar esse texto e identificar se ele trata de um caso de dano ambiental e extrair 13 informações específicas, conforme as instruções abaixo.
    Definição útil: Dano ambiental é qualquer prejuízo causado ao meio ambiente (água, solo, ar, fauna, flora, patrimônio paisagístico etc.) por ação ou omissão de um terceiro, sendo gerador de obrigação de reparação, conforme os artigos 186 e 927 do Código Civil.
    Retorno esperado (formato da resposta):
    A resposta deve ser apenas uma lista com 13 elementos, nesta ordem específica:

    [0] O texto trata de dano ambiental?
     Valores possíveis: True ou False

    [1] Trecho do texto que justifique a resposta acima.

    [2] Georreferência do local afetado, no formato: XX°xx’xx.xx” S e XX°xx’xx.xx” O
     Se não houver, retorne: "Não encontrado georreferencia"

    [3] Unidade Federativa (UF) completa (ex: "Maranhão", "São Paulo" etc.)
     Se não houver, retorne: "Não encontrado UF"

    [4] Município ou cidade do local afetado.
     Se não houver, retorne: "Não encontrado cidade"

    [5] Nome do responsável pelo dano ambiental (empresa ou pessoa física).
     Se não houver, retorne: "Não encontrado responsavel"

    [6] Categoria do responsável: "Pessoa Física" ou "Pessoa Jurídica"
     Se não houver, retorne: "Não encontrado pessoa"

    [7] Data do impacto ambiental no formato: DD/MM/AA
     Se não houver, retorne: "Não encontrado data"

    [8] Extensão da área afetada (ex: "120 hectares", "15.000 m²", etc.)
     Se não houver, retorne: "Não encontrado área"

    [9] Unidade de medida da área (ex: "hectares", "m²", etc.)
     Se não houver, retorne: "Não encontrado medida"

    [10] Metodologia de punição aplicada (ex: auto de infração, processo administrativo, etc.)
     Se não houver, retorne: "Não encontrado método"

    [11] Valor da multa, compensação monetária ou indenização atribuída (em reais, formato: R$ 000.000,00)
     Se não houver, retorne: "Não encontrado valor"

    [12] Compensação não monetária atribuída  (ou seja, alguma ação de reparação ambiental, como reflorestamento, recuperação de áreas degradadas, etc.)
     Se não houver, retorne: "Não encontrada compensação"

    IMPORTANTE: Não retorne o texto original, Não inclua explicações ou formatações extras, Apenas a lista com os 13 elementos na ordem acima, sem repetições. Não devolva o índice da lista, apenas os valores. Não inclua aspas ou colchetes na resposta. Se não encontrar algum elemento, retorne "Não encontrado" na posição correspondente da lista, apenas o valor da resposta.
    A resposta deve ser uma lista com EXATAMENTE 13 elementos (de índice 0 a 12), na ordem definida acima. Não retorne elementos extras. Não adicione vírgulas no final da lista. Não inclua o texto original, nem explicações, apenas a lista limpa.
    
    Texto bruto: {texto_extraido}"""
    generation_config_low_temp = genai.GenerationConfig(
        max_output_tokens=500,
        temperature=0.0,
)
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
    time.sleep(3)
    lista = (resposta.text.split("\n"))
    lista.insert(0,filtrados.iloc[i,1])
    lista = [x for x in lista if x != '']

    respostas[filtrados.iloc[i,2]] = lista
    

In [23]:
for i in range(0,50):
    analise(i,respostas_camila)
    time.sleep(3)
    #print(respostas_camila)

In [24]:
#print((list(respostas.keys())))
respostas_teste_camila = pd.DataFrame.from_dict(respostas_camila, orient='index')

respostas_teste_camila.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria']

In [25]:
respostas_teste_camila

,processoAnexoID,processo_ambiental,justificativa,georreferência,UF,municipio,responsavel,tipo_responsavel,data,área,unidade,metodologia_punição,valor_multa,compensacao_nao_monetaria
2643544408,652049379,False,O texto descreve um processo judicial sobre co...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Processo judicial,Não encontrado valor,Não encontrada compensação
2643544419,652049379,False,Não encontrado trecho,Não encontrado georreferencia,Maranhão,Não encontrado cidade,BRK AMBIENTAL - MARANHAO S.A,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,"R$ 15.000,00",Não encontrada compensação
2643544426,652049379,False,O texto trata de um processo judicial sobre co...,Não encontrado georreferencia,Maranhão,Paço do Lumiar,BRK Ambiental Maranhão S.A,Pessoa Jurídica,Não encontrado data,150,m²,Processo judicial,Não encontrado valor,Não encontrada compensação
590939495,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado área,Não encontrado medida,Processo judicial,Não encontrado valor,Plano de recuperação para conter a erosão que ...
590939496,612465137,True,Ação Civil Pública Cível referente a Área de P...,Não encontrado georreferencia,Não encontrado UF,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação
590939497,612465137,True,Foi realizada supressão de vegetação rasteira ...,20°10’19.74” S e 43°09’08.43” O,Minas Gerais,Ponte Nova,Wagner de Sousa das Neves,Pessoa Física,06/08/2020,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Plano de recuperação para conter a erosão que ...
1186042345,582564055,True,O processo trata de ação de reparação de danos...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Não encontrado responsavel,Não encontrado pessoa,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação
2589045924,690025576,True,O presente Laudo Técnico Pericial tem como obj...,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,CARMENSE COMERCIAL LTDA,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo administrativo,"R$ 3.807.896,40",Desativação e desmobilização completa do parqu...
2589045929,690025576,True,Mancha de óleo sobre o solo – Risco de contami...,Não encontrado georreferencia,Não encontrado UF,Não encontrado cidade,Carmense Comercial Ltda,Pessoa Jurídica,Não encontrado data,Não encontrado área,Não encontrado medida,Não encontrado método,Não encontrado valor,Não encontrada compensação
2589045772,690025576,True,ÁREA DE ATUAÇÃO: Meio Ambiente,Não encontrado georreferencia,Minas Gerais,Carmo da Mata,Carmense Comercial Ltda,Pessoa Jurídica,04/05/2023,Não encontrado área,Não encontrado medida,Processo administrativo,"R$ 2.447.933,38",Investigação Confirmatória em todo o parque in...


In [26]:
respostas_teste_camila.to_excel("testes_camila_1a50.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_2668\219740212.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_teste_camila.to_excel("testes_camila_1a50.xlsx")


## Nery

In [32]:
nery = {}
for i in range(50,103):
    analise(i,nery)
    time.sleep(2)

In [33]:
respostas_teste_nery = pd.DataFrame.from_dict(nery, orient='index')

respostas_teste_nery.columns = [
    "processoAnexoID",
    "processo_ambiental",
    "justificativa",
    "georreferência",  
    "UF",
    "municipio",
    "responsavel",
    "tipo_responsavel",
    "data",
    "área",
    "unidade",
    "metodologia_punição",
    "valor_multa",
    'compensacao_nao_monetaria']

In [34]:
respostas_teste_nery.to_excel("testes_nery_51a100.xlsx")

C:\Users\sergi\AppData\Local\Temp\ipykernel_2668\467845267.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  respostas_teste_nery.to_excel("testes_nery_51a100.xlsx")
